In [2]:
import tensorflow as tf

In [1]:
class ModelGen:
    def __init__(self):
        pass
    def generateModel(self, layers, session):
        self._s = session
        finalModel = []
        
        layerCount = 0
        for layer in layers:
            if layer[0] == 'conv':            
                finalModel.append(tf.layers.conv(inputs=finalModel[layerCount], filters=layer[1], kernel_size = [1,layer[2]], activation=layer[3]))
            elif layer[0] == 'dense':
                finalModel.append(tf.layers.dense(inputs=finalModel[layerCount], layer[1], activation=layer[2]))
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run(self.averageLoss, feed_dict={self.inputT : obs, self.priceChanges: prices}))
        print(prices)
        